<a href="https://colab.research.google.com/github/The-cheater/Deep_Learning_Models/blob/main/final_improvised.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Connect Google Drive
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import zipfile
import os

# Unzip GAF Images
with zipfile.ZipFile('/content/drive/MyDrive/dataset/GAF_Images.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/GAF_Images')

# Unzip MTF Images
with zipfile.ZipFile('/content/drive/MyDrive/dataset/MTF_Images.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/MTF_Images')

# Check extracted folders
print("GAF Images:", os.listdir('/content/GAF_Images')[:5])
print("MTF Images:", os.listdir('/content/MTF_Images')[:5])


GAF Images: ['GAF_Images']
MTF Images: ['MTF_Images']


In [ ]:
# ==========================
# 1️⃣ Imports & Config
# ==========================
import os
import numpy as np
from PIL import Image
import tensorflow as tf
from tensorflow.keras import layers, Model, Input

# Label mapping
label_map = {'EL': 0, 'PD': 1, 'S': 2}

# ==========================
# 2️⃣ Data Loader Functions
# ==========================
def parse_pair(gaf_path, mtf_path, label):
    # Load GAF
    gaf_img = tf.io.read_file(gaf_path)
    gaf_img = tf.image.decode_png(gaf_img, channels=3)
    gaf_img = tf.image.resize(gaf_img, [224, 224])
    gaf_img = tf.cast(gaf_img, tf.float32) / 255.0

    # Load MTF
    mtf_img = tf.io.read_file(mtf_path)
    mtf_img = tf.image.decode_png(mtf_img, channels=3)
    mtf_img = tf.image.resize(mtf_img, [224, 224])
    mtf_img = tf.cast(mtf_img, tf.float32) / 255.0

    return (gaf_img, mtf_img), label

def create_file_paths_and_labels(gaf_root, mtf_root):
    gaf_paths, mtf_paths, labels = [], [], []

    for root, _, files in os.walk(gaf_root):
        for file in files:
            if file.lower().endswith('_gaf.png'):
                gaf_path = os.path.join(root, file)
                mtf_path = gaf_path.replace('GAF_Images', 'MTF_Images').replace('_gaf.png', '_mtf.png')
                if not os.path.exists(mtf_path):
                    continue  # skip if no corresponding MTF file

                # Determine label
                parts = gaf_path.split(os.sep)
                for cn in label_map.keys():
                    if cn in parts:
                        label = label_map[cn]
                        break
                else:
                    continue

                gaf_paths.append(gaf_path)
                mtf_paths.append(mtf_path)
                labels.append(label)

    return gaf_paths, mtf_paths, labels

# ==========================
# 3️⃣ Dataset Pipeline
# ==========================
gaf_root = '/content/GAF_Images/GAF_Images'
mtf_root = '/content/MTF_Images/MTF_Images'

gaf_paths, mtf_paths, labels = create_file_paths_and_labels(gaf_root, mtf_root)
print(f"✅ Found {len(gaf_paths)} samples.")

batch_size = 16
dataset = tf.data.Dataset.from_tensor_slices((gaf_paths, mtf_paths, labels))
dataset = dataset.shuffle(buffer_size=10000)
dataset = dataset.map(lambda gaf, mtf, lbl: parse_pair(gaf, mtf, lbl), num_parallel_calls=tf.data.AUTOTUNE)
dataset = dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)

✅ Found 9064 samples.


In [ ]:
from tensorflow.keras import layers, Model, Input

def conv_block(x, filters):
    x = layers.Conv2D(filters, 3, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)
    return x

# Inputs
input_gaf = Input(shape=(224, 224, 3), name='gaf_input')
input_mtf = Input(shape=(224, 224, 3), name='mtf_input')

# GAF stream (4 blocks)
x1 = layers.Conv2D(64, 3, padding='same', activation='relu')(input_gaf)
x1 = conv_block(x1, 128)
x1 = conv_block(x1, 256)
x1 = conv_block(x1, 512)

# MTF stream (4 blocks)
x2 = layers.Conv2D(64, 3, padding='same', activation='relu')(input_mtf)
x2 = conv_block(x2, 128)
x2 = conv_block(x2, 256)
x2 = conv_block(x2, 512)

# Merge
merged = layers.Concatenate(axis=-1)([x1, x2])
merged = layers.Dropout(0.3)(merged)  # light regularization after merging

# Additional CNN blocks (matching diagram)
x = layers.Conv2D(512, 3, padding='same', activation='relu')(merged)
x = layers.BatchNormalization()(x)
x = layers.MaxPooling2D(pool_size=(2, 2))(x)

x = layers.Conv2D(512, 3, padding='same', activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.MaxPooling2D(pool_size=(2, 2))(x)

# 🟩 Replace Flatten with GlobalAveragePooling2D
x = layers.GlobalAveragePooling2D()(x)

# 🟩 Smaller dense layers (was 4096)
x = layers.Dense(512, activation='relu')(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(512, activation='relu')(x)
x = layers.Dropout(0.5)(x)

output = layers.Dense(3, activation='softmax')(x)  # 3 classes

model = Model(inputs=[input_gaf, input_mtf], outputs=output)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ gaf_input           │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ mtf_input           │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 224, 224,  │      1,792 │ gaf_input[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 224, 224,  │      1,792 │ mtf_input[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 224, 224,  │     73,856 │ conv2d[0][0]      │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_5 (Conv2D)   │ (None, 224, 224,  │     73,856 │ conv2d_4[0][0]    │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 224, 224,  │        512 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 224, 224,  │        512 │ conv2d_5[0][0]    │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 112, 112,  │          0 │ batch_normalizat… │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_3     │ (None, 112, 112,  │          0 │ batch_normalizat… │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 112, 112,  │    295,168 │ max_pooling2d[0]… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_6 (Conv2D)   │ (None, 112, 112,  │    295,168 │ max_pooling2d_3[… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 112, 112,  │      1,024 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 112, 112,  │      1,024 │ conv2d_6[0][0]    │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 56, 56,    │          0 │ batch_normalizat… │
│ (MaxPooling2D)      │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_4     │ (None, 56, 56,    │          0 │ batch_normalizat… │
│ (MaxPooling2D)      │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 56, 56,    │  1,180,160 │ max_pooling2d_1[

 Total params: 10,718,979 (40.89 MB)

 Trainable params: 10,713,347 (40.87 MB)

 Non-trainable params: 5,632 (22.00 KB)

In [ ]:
history = model.fit(dataset, epochs=30)

Epoch 1/30
567/567 ━━━━━━━━━━━━━━━━━━━━ 336s 523ms/step - accuracy: 0.5059 - loss: 1.2248
Epoch 2/30
567/567 ━━━━━━━━━━━━━━━━━━━━ 324s 478ms/step - accuracy: 0.5644 - loss: 0.9338
Epoch 3/30
567/567 ━━━━━━━━━━━━━━━━━━━━ 322s 478ms/step - accuracy: 0.5735 - loss: 0.8914
Epoch 4/30
567/567 ━━━━━━━━━━━━━━━━━━━━ 270s 476ms/step - accuracy: 0.5762 - loss: 0.8797
Epoch 5/30
567/567 ━━━━━━━━━━━━━━━━━━━━ 322s 476ms/step - accuracy: 0.5895 - loss: 0.8432
Epoch 6/30
567/567 ━━━━━━━━━━━━━━━━━━━━ 321s 475ms/step - accuracy: 0.6306 - loss: 0.7464
Epoch 7/30
567/567 ━━━━━━━━━━━━━━━━━━━━ 322s 476ms/step - accuracy: 0.6889 - loss: 0.6830
Epoch 8/30
567/567 ━━━━━━━━━━━━━━━━━━━━ 322s 477ms/step - accuracy: 0.7343 - loss: 0.5901
Epoch 9/30
567/567 ━━━━━━━━━━━━━━━━━━━━ 322s 476ms/step - accuracy: 0.7836 - loss: 0.4860
Epoch 10/30
567/567 ━━━━━━━━━━━━━━━━━━━━ 322s 476ms/step - accuracy: 0.8454 - loss: 0.3815
Epoch 11/30
567/567 ━━━━━━━━━━━━━━━━━━━━ 269s 475ms/step - accuracy: 0.8876 - loss: 0.2854
Epoch 12

In [ ]:
model.save('/content/drive/MyDrive/my_model.h5')

# Or, save as a checkpoint
model.save_weights('/content/drive/MyDrive/my_model_weights.h5')

NameError: name 'model' is not defined

In [ ]:
from tensorflow import keras

# Load the saved model
model = keras.models.load_model('/content/drive/MyDrive/my_model.h5')


FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = '/content/drive/MyDrive/my_model.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
# 📌 Upload Images from Local Machine
from google.colab import files
from IPython.display import display
from PIL import Image
import tensorflow as tf
import numpy as np

print("✅ Please upload the GAF image (RGB)")
uploaded_gaf = files.upload()
gaf_path = list(uploaded_gaf.keys())[0]

print("✅ Please upload the MTF image (RGB)")
uploaded_mtf = files.upload()
mtf_path = list(uploaded_mtf.keys())[0]

# Display the images
display(Image.open(gaf_path))
display(Image.open(mtf_path))

# 🖼️ Preprocessing
def preprocess_image(img_path):
    img = Image.open(img_path).convert('RGB')
    img = img.resize((224, 224))
    img_array = np.array(img) / 255.0
    return np.expand_dims(img_array, axis=0)

gaf_test = preprocess_image(gaf_path)
mtf_test = preprocess_image(mtf_path)

print("✅ Preprocessed shapes:", gaf_test.shape, mtf_test.shape)

# ✅ Load the trained model
model = tf.keras.models.load_model('/content/mmcnn_model_final.h5')
print("✅ Model loaded.")

# 🔮 Predict
predictions = model.predict([gaf_test, mtf_test])
predicted_class = np.argmax(predictions, axis=1)[0]
confidence = np.max(predictions)

# Mapping back to class name
class_map = {0: 'EL', 1: 'PD', 2: 'S'}
print(f"✅ Predicted Class: {class_map[predicted_class]} with Confidence: {confidence:.4f}")


✅ Please upload the GAF image (RGB)
